In [36]:
project_name = 'full_fine_tuning_50epochs_final1'

datas=( 'stanford_cars', 'caltech101', 'fgvc-aircraft-2013b', 'dtd', 'flowers-102', 'oxford-iiit-pet'  ) #'imagenette2' 'eurosat' 

losses=( 'TRADES_v2', 'CLASSIC_AT' ) #  

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k'  ) 

import pickle
import numpy as np

final_data = []

for backbone in backbones:
    for loss in losses:
        for data in datas:

            try:

                name ='{}_{}_{}'.format(backbone, data, loss)
                with open('./results/results_{}_{}_{}_{}.pkl'.format(project_name, backbone, data, loss), 'rb') as f:
                    saved_data = pickle.load(f)
                    result = saved_data[name]["statistics"]
                    result['backbone'] = backbone
                    result['dataset'] = data
                    result['loss_function'] = loss    

                    print(name, saved_data[name]["statistics"])
            except:
                result = {}
                result['backbone'] = backbone
                result['dataset'] = data
                result['loss_function'] = loss   
                result['clean_acc'] = np.nan
                result['robust_acc'] = np.nan 
                # print('not available')

            final_data.append(result)

convnext_tiny_stanford_cars_TRADES_v2 {'clean_acc': 0.7666666666666667, 'robust_acc': 0.5133333333333333, 'backbone': 'convnext_tiny', 'dataset': 'stanford_cars', 'loss_function': 'TRADES_v2'}
convnext_tiny_caltech101_TRADES_v2 {'clean_acc': 0.9706666666666667, 'robust_acc': 0.906, 'backbone': 'convnext_tiny', 'dataset': 'caltech101', 'loss_function': 'TRADES_v2'}
convnext_tiny_fgvc-aircraft-2013b_TRADES_v2 {'clean_acc': 0.6653333333333333, 'robust_acc': 0.35733333333333334, 'backbone': 'convnext_tiny', 'dataset': 'fgvc-aircraft-2013b', 'loss_function': 'TRADES_v2'}
convnext_tiny_dtd_TRADES_v2 {'clean_acc': 0.7326666666666667, 'robust_acc': 0.46, 'backbone': 'convnext_tiny', 'dataset': 'dtd', 'loss_function': 'TRADES_v2'}
convnext_tiny_flowers-102_TRADES_v2 {'clean_acc': 0.9553333333333334, 'robust_acc': 0.8153333333333334, 'backbone': 'convnext_tiny', 'dataset': 'flowers-102', 'loss_function': 'TRADES_v2'}
convnext_tiny_stanford_cars_CLASSIC_AT {'clean_acc': 0.6873333333333334, 'robus

In [37]:
import pandas as pd

df = pd.DataFrame(final_data)
# df.to_csv('results.csv')
df

,clean_acc,robust_acc,backbone,dataset,loss_function
0,0.766667,0.513333,convnext_tiny,stanford_cars,TRADES_v2
1,0.970667,0.906000,convnext_tiny,caltech101,TRADES_v2
2,0.665333,0.357333,convnext_tiny,fgvc-aircraft-2013b,TRADES_v2
3,0.732667,0.460000,convnext_tiny,dtd,TRADES_v2
4,0.955333,0.815333,convnext_tiny,flowers-102,TRADES_v2
...,...,...,...,...,...
199,NaN,NaN,vit_base_patch16_224_miil.in21k,caltech101,CLASSIC_AT
200,0.022667,0.010000,vit_base_patch16_224_miil.in21k,fgvc-aircraft-2013b,CLASSIC_AT
201,0.378667,0.263333,vit_base_patch16_224_miil.in21k,dtd,CLASSIC_AT
202,0.394667,0.338667,vit_base_patch16_224_miil.in21k,flowers-102,CLASSIC_AT


In [38]:
data_updated = df.copy()

# Combine clean_acc and robust_acc as independent metrics for ranking
data_updated['total_acc'] = data_updated['clean_acc'] + data_updated['robust_acc']  # Sum of scores per dataset

# Aggregate total_acc across all datasets for each backbone + loss combination
aggregated_data_no_avg = data_updated.groupby(['backbone', 'loss_function']).agg(
    total_sum=('total_acc', 'sum'),  # Total sum of all scores across datasets
    total_geom_mean=('total_acc', lambda x: np.prod(x) ** (1 / len(x))),  # Geometric mean
).reset_index()

# Rank based on these combined scores
aggregated_data_no_avg['rank_total_sum'] = aggregated_data_no_avg['total_sum'].rank(ascending=False)
aggregated_data_no_avg['rank_total_geom_mean'] = aggregated_data_no_avg['total_geom_mean'].rank(ascending=False)

# Sort by sum ranking for presentation
ranked_aggregated_data_no_avg = aggregated_data_no_avg.sort_values(by='rank_total_sum')

ranked_aggregated_data_no_avg

# tools.display_dataframe_to_user(name="Ranked Backbone and Loss Combinations Without Averaging", dataframe=ranked_aggregated_data_no_avg)


,backbone,loss_function,total_sum,total_geom_mean,rank_total_sum,rank_total_geom_mean
16,robust_convnext_tiny,CLASSIC_AT,8.363333,1.359192,1.000000,3.000000
24,vit_base_patch16_224.dino,CLASSIC_AT,7.732667,1.241263,2.000000,13.000000
11,convnext_tiny.fb_in22k,TRADES_v2,7.680667,1.413271,3.000000,1.000000
18,robust_deit_small_patch16_224,CLASSIC_AT,7.612667,1.222360,4.000000,15.000000
10,convnext_tiny.fb_in22k,CLASSIC_AT,7.447333,1.009502,5.000000,30.000000
8,convnext_tiny,CLASSIC_AT,7.374667,1.185441,6.000000,22.000000
29,vit_base_patch16_224.orig_in21k,TRADES_v2,7.362000,1.356705,7.000000,4.000000
17,robust_convnext_tiny,TRADES_v2,7.359333,1.363603,8.000000,2.000000
31,vit_base_patch16_224.sam_in1k,TRADES_v2,7.246667,1.343138,9.000000,5.000000
33,vit_base_patch16_224_miil.in21k,TRADES_v2,7.208000,1.099455,10.000000,26.000000


In [39]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=[ 'clean_acc',  ], #'robust_acc'
    dropna=False
)

grouped_df #.to_csv('results_acc.csv' )

clean_acc           \
dataset                                         caltech101      dtd   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT           NaN 0.714000   
                                  TRADES_v2            NaN 0.720667   
convnext_base.clip_laion2b        CLASSIC_AT           NaN 0.689333   
                                  TRADES_v2            NaN 0.706000   
convnext_base.clip_laion2b_augreg CLASSIC_AT           NaN 0.720667   
                                  TRADES_v2            NaN 0.782667   
convnext_base.fb_in22k            CLASSIC_AT           NaN 0.734000   
                                  TRADES_v2            NaN 0.730000   
convnext_tiny                     CLASSIC_AT      0.964000 0.707333   
                                  TRADES_v2       0.970667 0.732667   
convnext_tiny.fb_in22k            CLASSIC_AT      0.976667 0.689333   
                                  TRADES_v2       0.968667 0.724000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.956667 0.646667   
                                  TRADES_v2       0.958667 0.644667   
robust_convnext_base              CLASSIC_AT           NaN 0.593333   
                                  TRADES_v2            NaN 0.639333   
robust_convnext_tiny              CLASSIC_AT      0.964667 0.611333   
                                  TRADES_v2       0.972667 0.655333   
robust_deit_small_patch16_224     CLASSIC_AT      0.948000 0.576000   
                                  TRADES_v2       0.961333 0.603333   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.932667 0.561333   
                                  TRADES_v2       0.960667 0.663333   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.929333 0.625333   
                                  TRADES_v2       0.977333 0.726000   
vit_base_patch16_224.dino         CLASSIC_AT      0.960667 0.584667   
                                  TRADES_v2       0.963333 0.700667   
vit_base_patch16_224.mae          CLASSIC_AT           NaN 0.524667   
                                  TRADES_v2       0.894667 0.650000   
vit_base_patch16_224.orig_in21k   CLASSIC_AT           NaN 0.610000   
                                  TRADES_v2       0.972667 0.723333   
vit_base_patch16_224.sam_in1k     CLASSIC_AT           NaN 0.562667   
                                  TRADES_v2       0.968000 0.677333   
vit_base_patch16_224_miil.in21k   CLASSIC_AT           NaN 0.378667   
                                  TRADES_v2       0.956667 0.664667   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.526667   
                                  TRADES_v2                0.751333   
convnext_base.clip_laion2b        CLASSIC_AT               0.732000   
                                  TRADES_v2                0.776000   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.745333   
                                  TRADES_v2                0.752667   
convnext_base.fb_in22k            CLASSIC_AT               0.516667   
                                  TRADES_v2                0.771333   
convnext_tiny                     CLASSIC_AT               0.591333   
                                  TRADES_v2                0.665333   
convnext_tiny.fb_in22k            CLASSIC_AT               0.150667   
                                  TRADES_v2                0.757333   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.491333   
                                  TRADES_v2                0.634000   
robust_convnext_base              CLASSIC_AT               0.679333   
                                  TRADES_v2                0.732000   
robust_convnext_tiny              CLASSIC_AT               0.537333   
 

In [40]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index
grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'], 
    columns='dataset', 
    values=['robust_acc', ], #
    dropna=False
)

grouped_df

robust_acc           \
dataset                                         caltech101      dtd   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT           NaN 0.516667   
                                  TRADES_v2            NaN 0.536667   
convnext_base.clip_laion2b        CLASSIC_AT           NaN 0.469333   
                                  TRADES_v2            NaN 0.490667   
convnext_base.clip_laion2b_augreg CLASSIC_AT           NaN 0.495333   
                                  TRADES_v2            NaN 0.496000   
convnext_base.fb_in22k            CLASSIC_AT           NaN 0.531333   
                                  TRADES_v2            NaN 0.527333   
convnext_tiny                     CLASSIC_AT      0.903333 0.390000   
                                  TRADES_v2       0.906000 0.460000   
convnext_tiny.fb_in22k            CLASSIC_AT      0.916000 0.472000   
                                  TRADES_v2       0.922000 0.502000   
deit_small_patch16_224.fb_in1k    CLASSIC_AT      0.874667 0.308667   
                                  TRADES_v2       0.880667 0.401333   
robust_convnext_base              CLASSIC_AT           NaN 0.479333   
                                  TRADES_v2            NaN 0.520000   
robust_convnext_tiny              CLASSIC_AT      0.930667 0.478667   
                                  TRADES_v2       0.941333 0.502000   
robust_deit_small_patch16_224     CLASSIC_AT      0.903333 0.415333   
                                  TRADES_v2       0.920667 0.430667   
vit_base_patch16_224.augreg_in1k  CLASSIC_AT      0.858000 0.369333   
                                  TRADES_v2       0.881333 0.414000   
vit_base_patch16_224.augreg_in21k CLASSIC_AT      0.887333 0.412667   
                                  TRADES_v2       0.920667 0.388667   
vit_base_patch16_224.dino         CLASSIC_AT      0.895333 0.370000   
                                  TRADES_v2       0.886000 0.396667   
vit_base_patch16_224.mae          CLASSIC_AT           NaN 0.320000   
                                  TRADES_v2       0.799333 0.382667   
vit_base_patch16_224.orig_in21k   CLASSIC_AT           NaN 0.376000   
                                  TRADES_v2       0.903333 0.440667   
vit_base_patch16_224.sam_in1k     CLASSIC_AT           NaN 0.364000   
                                  TRADES_v2       0.892667 0.438000   
vit_base_patch16_224_miil.in21k   CLASSIC_AT           NaN 0.263333   
                                  TRADES_v2       0.899333 0.398667   

                                                                     \
dataset                                         fgvc-aircraft-2013b   
backbone                          loss_function                       
convnext_base                     CLASSIC_AT               0.350667   
                                  TRADES_v2                0.627333   
convnext_base.clip_laion2b        CLASSIC_AT               0.574667   
                                  TRADES_v2                0.500667   
convnext_base.clip_laion2b_augreg CLASSIC_AT               0.582667   
                                  TRADES_v2                0.622000   
convnext_base.fb_in22k            CLASSIC_AT               0.336667   
                                  TRADES_v2                0.627333   
convnext_tiny                     CLASSIC_AT               0.392667   
                                  TRADES_v2                0.357333   
convnext_tiny.fb_in22k            CLASSIC_AT               0.048667   
                                  TRADES_v2                0.600667   
deit_small_patch16_224.fb_in1k    CLASSIC_AT               0.259333   
                                  TRADES_v2                0.456000   
robust_convnext_base              CLASSIC_AT               0.606667   
                                  TRADES_v2                0.642667   
robust_convnext_tiny              CLASSIC_AT               0.430000   


In [26]:
import pandas as pd
import numpy as np
import plotly.express as px


df_new_cleaned = df #.drop(columns=['Unnamed: 0'])
name =  'TRADES_v2' #
df_new_cleaned = df_new_cleaned[ df_new_cleaned.loss_function != name ]

# Assuming df_new_cleaned already exists and contains all necessary columns
# Group by backbone and loss function, compute the mean and standard deviation for clean and robust accuracy
aggregated_metrics = df_new_cleaned.groupby(['backbone', 'loss_function']).agg(
    Mean_Clean_Acc=('clean_acc', 'mean'),
    Std_Clean_Acc=('clean_acc', 'std'),
    Mean_Robust_Acc=('robust_acc', 'mean'),
    Std_Robust_Acc=('robust_acc', 'std')
).reset_index()

# Classify architectures into categories
aggregated_metrics['Architecture'] = aggregated_metrics['backbone'].apply(
    lambda name: 'ConvNext' if 'convnext' in name.lower() else ('ViT/DeiT' if 'deit' in name.lower() or 'vit' in name.lower() else 'Other')
)

# Assign marker shapes based on loss function
aggregated_metrics['Marker_Shape'] = aggregated_metrics['loss_function'].apply(
    lambda loss: 'circle' if loss == 'TRADES_v2' else ('square' if loss == 'CLASSIC_AT' else 'diamond')
)

# Identify Pareto front
x = aggregated_metrics['Mean_Clean_Acc']
y = aggregated_metrics['Mean_Robust_Acc']
is_pareto = np.array([
    not any((y[j] >= y[i] and x[j] >= x[i]) for j in range(len(x)) if i != j)
    for i in range(len(x))
])
pareto_points = aggregated_metrics[is_pareto]

# Add a mapping for loss functions to valid Plotly symbols
symbol_mapping = {
    'TRADES_v2': 'circle',
    'CLASSIC_AT': 'square'
}

# Update the scatter plot with proper legend entries for loss functions
fig = px.scatter(
    aggregated_metrics,
    x='Mean_Clean_Acc',
    y='Mean_Robust_Acc',
    color='Architecture',         # Color for architecture
    symbol='loss_function',       # Symbol for loss function
    # title='Backbone Performance: Clean vs. Robust Accuracy',
    labels={
        'Mean_Clean_Acc': 'Mean Clean Accuracy',
        'Mean_Robust_Acc': 'Mean Robust Accuracy',
        'loss_function': 'Loss Function'  # Custom label for legend
    },
    color_discrete_map={
        'ConvNext': '#1f77b4',  # Blue for ConvNext
        'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
        'Other': '#2ca02c'      # Green for Other architectures
    },
    symbol_map=symbol_mapping    # Apply symbol mapping
)

# Add static backbone names as labels on the figure
for i, row in aggregated_metrics.iterrows():
    fig.add_annotation(
        x=row['Mean_Clean_Acc'] - 0.01,
        y=row['Mean_Robust_Acc'] + 0.01,  # Offset text slightly above the marker
        text=row['backbone'],
        showarrow=False,
        font=dict(size=10),
        align='center'
    )

# Add Pareto front markers with larger size and same shape/color as the original
for idx, d in enumerate(pareto_points.iterrows()):
    _, row = d
    fig.add_scatter(
        x=[row['Mean_Clean_Acc']],
        y=[row['Mean_Robust_Acc']],
        mode='markers',
        showlegend=False,  # Avoid duplicating legends
        marker=dict(
            size=13,  # Larger marker size
            symbol=symbol_mapping[row['loss_function']],  # Use the mapped symbol
            color=aggregated_metrics.loc[aggregated_metrics['backbone'] == row['backbone'], 'Architecture'].map({
                'ConvNext': '#1f77b4',  # Blue for ConvNext
                'ViT/DeiT': '#ff7f0e',  # Orange for ViT/DeiT
                'Other': '#2ca02c'     # Green for Other architectures
            }).values[0]  # Map the color dynamically
        ),
        name=f"Pareto"
    )


# Add a diagonal line (x = y)
fig.add_shape(
    type="line",
    x0=0, y0=0,  # Start of line
    x1=1, y1=1,  # End of line
    xref="paper", yref="paper",  # Use normalized coordinates (0 to 1 in plot space)
    line=dict(color="red", width=2, dash="dot")  # Line style
)

# Adjust layout for better visual clarity
fig.update_layout(
    legend_title=dict(text='Backbone and Loss Function'),
    width=600,
    height=600,
    font=dict(size=12),
    #title_font=dict(size=14),
    margin=dict(l=10, r=45, t=10, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    xaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', zeroline=False),
    yaxis_range=[0,0.7],
    xaxis_range=[0,0.85]
)


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85 ]
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ],
        ticktext=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 ]
    )
)

fig.show()
fig.write_image( "./figures/{}_{}.png".format(project_name,name) )

In [27]:
import comet_ml
import os

os.environ["COMET_API_KEY"] = "I5AiXfuD0TVuSz5UOtujrUM9i"

PROJECT_NAME="full-fine-tuning-5epochs-final1"
WORKSPACE_NAME="maxheuillet"

# Get all Experiemnts in workspace
api = comet_ml.API()

# all_experiments_in_workspace = api.get_experiments(WORKSPACE_NAME)
all_experiments_in_project = api.get_experiments(WORKSPACE_NAME, PROJECT_NAME)


backbones = [
  "convnext_tiny", "robust_convnext_tiny", "convnext_tiny.fb_in22k", 
  "deit_small_patch16_224.fb_in1k", "robust_deit_small_patch16_224",
  "convnext_base", "convnext_base.fb_in22k", "robust_convnext_base",
  "convnext_base.clip_laion2b", "convnext_base.clip_laion2b_augreg",
  "vit_base_patch16_224.augreg_in1k", "vit_base_patch16_224.augreg_in21k",
  "vit_base_patch16_224.dino", "vit_base_patch16_224.mae", 
  "vit_base_patch16_224.orig_in21k", "vit_base_patch16_224.sam_in1k",
  "vit_base_patch16_224_miil.in21k", ]

datas = [ "stanford_cars", "caltech101", "fgvc-aircraft-2013b", 
          "dtd", "flowers-102", "oxford-iiit-pet", ]

losses = [ "TRADES_v2", "CLASSIC_AT", ]

# 1) Build a dictionary
experiment_map = {}
for b in backbones:
    for d in datas:
        for l in losses:
            exp_name = f"{b}_{d}_{l}"
            experiment_map[exp_name] = (b, d, l)

In [28]:
import tqdm.notebook as tqdm
import pandas as pd

# data = all_experiments_in_project[0].get_metrics()
# import pandas as pd
# data = pd.DataFrame(data)
# set(data.metricName)

dataframe = []

for i in range(len(all_experiments_in_project)) :

    # 2) Suppose we have an experiment name to parse
    experiment_name = all_experiments_in_project[i].get_name()

    # 3) Look it up
    if experiment_name in experiment_map:
        backbone, dataset, loss = experiment_map[experiment_name]
        print("Backbone:", backbone)
        print("Dataset:", dataset)
        print("Loss:", loss)
    else:
        print("Experiment name not found in known combinations!")

    data = all_experiments_in_project[i].get_metrics()

    data = pd.DataFrame(data)

    df_filtered = data[ data.metricName == 'zero_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    zero_adv_val = df_filtered["metricValue"].tolist()
    zero_adv_val = [float(x) for x in zero_adv_val]

    df_filtered = data[ data.metricName == 'dormant_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    dormant_adv_val = df_filtered["metricValue"].tolist()
    dormant_adv_val = [float(x) for x in dormant_adv_val]

    df_filtered = data[ data.metricName == 'overactive_adv_val' ]
    df_filtered = df_filtered.sort_values("step")
    overactive_adv_val = df_filtered["metricValue"].tolist()
    overactive_adv_val = [float(x) for x in overactive_adv_val]


    df_filtered = data[ data.metricName == 'zero_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    zero_adv_train = df_filtered["metricValue"].tolist()
    zero_adv_train = [float(x) for x in zero_adv_train]

    df_filtered = data[ data.metricName == 'dormant_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    dormant_adv_train = df_filtered["metricValue"].tolist()
    dormant_adv_train = [float(x) for x in dormant_adv_train]

    df_filtered = data[ data.metricName == 'overactive_adv_train' ]
    df_filtered = df_filtered.sort_values("step")
    overactive_adv_train = df_filtered["metricValue"].tolist()
    overactive_adv_train = [float(x) for x in overactive_adv_train]

    df_filtered = data[ data.metricName == 'gradient_norm' ]
    df_filtered = df_filtered.sort_values("step")
    gradient_norm = df_filtered["metricValue"].tolist()
    gradient_norm = [float(x) for x in gradient_norm]

    if len(zero_adv_train) != 0:

        result = {'backbone':backbone, 'loss_function':loss, 'dataset':dataset,
                
            'zero_adv_train0':zero_adv_train[0], 
            'dormant_adv_train0':dormant_adv_train[0], 
            'overactive_adv_train0':overactive_adv_train[0], 
                
            'zero_adv_train5':zero_adv_train[-1], 
            'dormant_adv_train5':dormant_adv_train[-1], 
            'overactive_adv_train5':overactive_adv_train[-1],

            'zero_adv_val0':zero_adv_val[0], 
            'dormant_adv_val0':dormant_adv_val[0], 
            'overactive_adv_val0':overactive_adv_val[0], 
                
            'zero_adv_val5':zero_adv_val[-1], 
            'dormant_adv_val5':dormant_adv_val[-1], 
            'overactive_adv_val5':overactive_adv_val[-1], }
        
    else:

        pass

        # result = {'backbone':backbone, 'loss_function':loss, 'dataset':dataset,
                
        #         'zero_adv_train0':np.nan, 
        #         'dormant_adv_train0':np.nan, 
        #         'overactive_adv_train0':np.nan, 
                
        #         'zero_adv_train5':np.nan, 
        #         'dormant_adv_train5':np.nan, 
        #         'overactive_adv_train5':np.nan,  

        #         'zero_adv_val0':np.nan, 
        #         'dormant_adv_val0':np.nan, 
        #         'overactive_adv_val0':np.nan, 
                    
        #         'zero_adv_val5':np.nan, 
        #         'dormant_adv_val5':np.nan,  
        #         'overactive_adv_val5':np.nan,   }
    
    dataframe.append(result)

    # 'gradient_norm5':gradient_norm,
    # 'gradient_norm0':gradient_norm,

Backbone: robust_convnext_tiny
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_tiny.fb_in22k
Dataset: dtd
Loss: TRADES_v2
Backbone: deit_small_patch16_224.fb_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: robust_deit_small_patch16_224
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.sam_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base
Dataset: dtd
Loss: TRADES_v2
Backbone: robust_convnext_base
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.fb_in22k
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.clip_laion2b
Dataset: dtd
Loss: TRADES_v2
Backbone: convnext_base.clip_laion2b_augreg
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.augreg_in1k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.mae
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.orig_in21k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.augreg_in21k
Dataset: dtd
Loss: TRADES_v2
Backbone: vit_base_patch16_224.dino
Dataset: dtd
Loss: TRADES_v2
Backbone: v

In [29]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr

df2 = pd.DataFrame(dataframe)

df_merged = pd.merge(df, df2, on=["backbone", "loss_function", "dataset"], how="inner")

df_merged = df_merged.dropna(subset=["clean_acc", "robust_acc"])

df_merged.to_csv('results_corr.csv')

# Define a function to calculate correlation and p-values for a set of columns
def compute_correlations_with_pvalues(df, cols, target_vars):
    results = []
    
    for target in target_vars:  # clean_acc and robust_acc
        for col in cols:
            # Compute Pearson correlation and p-value

            # print( df[target], df[col] )
            pearson_corr, pearson_p = pearsonr(df[target], df[col])
            # Compute Spearman correlation and p-value
            spearman_corr, spearman_p = spearmanr(df[target], df[col])
            
            results.append({
                "target": target,
                "feature": col,
                "pearson_corr": pearson_corr,
                "pearson_pval": pearson_p,
                "spearman_corr": spearman_corr,
                "spearman_pval": spearman_p
            })
    
    return pd.DataFrame(results)

# Compute correlations and p-values for the specified columns
cols = [
    "zero_adv_train0",
    "dormant_adv_train0",
    "overactive_adv_train0",
    "zero_adv_train5",
    "dormant_adv_train5",
    "overactive_adv_train5",
    "zero_adv_val0",
    "dormant_adv_val0",
    "overactive_adv_val0",
]

target_vars = ["clean_acc", "robust_acc"]

# Correlation with p-values for the entire dataset
correlation_pvalues = compute_correlations_with_pvalues(df_merged, cols, target_vars)
# Set display option
pd.set_option("display.float_format", "{:.6f}".format)

correlation_pvalues

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.052696,0.360652,-0.015379,0.789768
1,clean_acc,dormant_adv_train0,-0.228202,0.000061,-0.217585,0.000135
2,clean_acc,overactive_adv_train0,-0.295619,0.000000,-0.264494,0.000003
3,clean_acc,zero_adv_train5,-0.055482,0.335792,-0.065050,0.258969
4,clean_acc,dormant_adv_train5,-0.309784,0.000000,-0.261984,0.000004
5,clean_acc,overactive_adv_train5,-0.295172,0.000000,-0.284226,0.000000
6,clean_acc,zero_adv_val0,-0.108619,0.058960,-0.062841,0.275527
7,clean_acc,dormant_adv_val0,-0.371039,0.000000,-0.286809,0.000000
8,clean_acc,overactive_adv_val0,-0.311086,0.000000,-0.275345,0.000001
9,robust_acc,zero_adv_train0,0.019196,0.739293,-0.032260,0.575912


In [30]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

# Define the column groups for analysis
cols = [
    "zero_adv_train0",
    "dormant_adv_train0",
    "overactive_adv_train0",
    "zero_adv_train5",
    "dormant_adv_train5",
    "overactive_adv_train5",
    "zero_adv_val0",
    "dormant_adv_val0",
    "overactive_adv_val0",
]

# Define the backbone and loss groupings
group_backbones = {
    "convnext": ['convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k',
                 'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base',
                 'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',  ],

    "vit": ['deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
            'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
            'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
            'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k' ]
}

group_losses = {
    "trades_v2": ["TRADES_v2"],
    "classic_at": ["CLASSIC_AT"]
}

# Function to compute Pearson and Spearman correlations
def compute_group_correlations_and_summary(df, group_filter, target_cols):
    subgroup = df[group_filter]
    correlations = []
    summary_stats = []

    for target_col in target_cols:
        for acc_col in ["clean_acc", "robust_acc"]:
            # Drop NaNs for valid correlation computation
            valid_data = subgroup[[acc_col, target_col]].dropna()

            # Compute correlations if enough data
            if len(valid_data) > 1:
                # Pearson correlation
                pearson_corr, pearson_pval = pearsonr(valid_data[acc_col], valid_data[target_col])
                # Spearman correlation
                spearman_corr, spearman_pval = spearmanr(valid_data[acc_col], valid_data[target_col])
            else:
                pearson_corr = pearson_pval = spearman_corr = spearman_pval = float("nan")

            correlations.append({
                "target": acc_col,
                "feature": target_col,
                "pearson_corr": pearson_corr,
                "pearson_pval": pearson_pval,
                "spearman_corr": spearman_corr,
                "spearman_pval": spearman_pval
            })



    return pd.DataFrame(correlations)

# Create group filters
group_filters = {
    "convnext_trades_v2": (df_merged["backbone"].isin(group_backbones["convnext"])) & (df_merged["loss_function"].isin(group_losses["trades_v2"])),
    "convnext_classic_at": (df_merged["backbone"].isin(group_backbones["convnext"])) & (df_merged["loss_function"].isin(group_losses["classic_at"])),
    "vit_trades_v2": (df_merged["backbone"].isin(group_backbones["vit"])) & (df_merged["loss_function"].isin(group_losses["trades_v2"])),
    "vit_classic_at": (df_merged["backbone"].isin(group_backbones["vit"])) & (df_merged["loss_function"].isin(group_losses["classic_at"]))
}

# Compute correlations and summary statistics for each group
group_results = {}
for group_name, group_filter in group_filters.items():
    correlations = compute_group_correlations_and_summary(df_merged, group_filter, cols)
    group_results[group_name] =  correlations
       




In [31]:
group_results['convnext_trades_v2']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.278077,0.014337,0.193150,0.092365
1,robust_acc,zero_adv_train0,0.176726,0.124164,0.129389,0.262061
2,clean_acc,dormant_adv_train0,-0.173049,0.132313,0.105029,0.363313
3,robust_acc,dormant_adv_train0,0.002815,0.980615,0.104542,0.365557
4,clean_acc,overactive_adv_train0,-0.084981,0.462439,-0.043963,0.704207
5,robust_acc,overactive_adv_train0,0.044153,0.702990,-0.018587,0.872526
6,clean_acc,zero_adv_train5,0.366923,0.001029,0.277327,0.014613
7,robust_acc,zero_adv_train5,0.288635,0.010904,0.227680,0.046432
8,clean_acc,dormant_adv_train5,0.029108,0.801586,0.142149,0.217493
9,robust_acc,dormant_adv_train5,0.055022,0.634588,0.078625,0.496697


In [32]:
group_results['convnext_classic_at']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,0.023267,0.847277,-0.052414,0.664208
1,robust_acc,zero_adv_train0,-0.049247,0.683386,-0.056147,0.641879
2,clean_acc,dormant_adv_train0,-0.019678,0.870609,0.056650,0.638895
3,robust_acc,dormant_adv_train0,0.160410,0.181444,0.165989,0.166530
4,clean_acc,overactive_adv_train0,-0.136975,0.254675,-0.120374,0.317344
5,robust_acc,overactive_adv_train0,0.036949,0.759670,0.015041,0.900923
6,clean_acc,zero_adv_train5,0.026053,0.829249,-0.071639,0.552718
7,robust_acc,zero_adv_train5,0.012887,0.915053,-0.077860,0.518673
8,clean_acc,dormant_adv_train5,-0.189723,0.113037,-0.119617,0.320420
9,robust_acc,dormant_adv_train5,-0.161254,0.179128,-0.093539,0.437815


In [33]:
group_results['vit_trades_v2']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,-0.021508,0.851733,-0.156328,0.171694
1,robust_acc,zero_adv_train0,0.016650,0.884961,-0.062502,0.586701
2,clean_acc,dormant_adv_train0,-0.122835,0.283990,-0.221605,0.051190
3,robust_acc,dormant_adv_train0,-0.020020,0.861886,-0.062945,0.584047
4,clean_acc,overactive_adv_train0,-0.364979,0.001018,-0.370253,0.000848
5,robust_acc,overactive_adv_train0,-0.341184,0.002236,-0.336946,0.002557
6,clean_acc,zero_adv_train5,-0.190859,0.094163,-0.240458,0.033955
7,robust_acc,zero_adv_train5,-0.139728,0.222420,-0.159868,0.162074
8,clean_acc,dormant_adv_train5,-0.387254,0.000461,-0.364255,0.001044
9,robust_acc,dormant_adv_train5,-0.259408,0.021822,-0.235139,0.038236


In [34]:
group_results['vit_classic_at']

,target,feature,pearson_corr,pearson_pval,spearman_corr,spearman_pval
0,clean_acc,zero_adv_train0,-0.073003,0.528062,-0.008945,0.938456
1,robust_acc,zero_adv_train0,-0.061549,0.594893,-0.017415,0.880505
2,clean_acc,dormant_adv_train0,-0.235505,0.039223,-0.250567,0.027953
3,robust_acc,dormant_adv_train0,-0.187197,0.103054,-0.205036,0.073644
4,clean_acc,overactive_adv_train0,-0.271727,0.016822,-0.172416,0.133756
5,robust_acc,overactive_adv_train0,-0.248572,0.029267,-0.212981,0.062923
6,clean_acc,zero_adv_train5,-0.341039,0.002404,-0.211748,0.064499
7,robust_acc,zero_adv_train5,-0.280173,0.013589,-0.223701,0.050498
8,clean_acc,dormant_adv_train5,-0.358839,0.001352,-0.328837,0.003500
9,robust_acc,dormant_adv_train5,-0.312731,0.005621,-0.295269,0.009134


In [35]:
import pandas as pd

# Function to compute summary statistics for clean_acc and robust_acc
def compute_summary_statistics(df, group_filter):
    subgroup = df[group_filter]
    summary_stats = []

    for acc_col in ["clean_acc", "robust_acc"]:
        # Compute statistics
        summary_stats.append({
            "metric": acc_col,
            "mean": subgroup[acc_col].mean(),
            "std_dev": subgroup[acc_col].std(),
            "min": subgroup[acc_col].min(),
            "max": subgroup[acc_col].max(),
            "count": subgroup[acc_col].count()
        })

    return pd.DataFrame(summary_stats)

# Compute statistics for each group
group_stats = {}
for group_name, group_filter in group_filters.items():
    stats = compute_summary_statistics(df_merged, group_filter)
    group_stats[group_name] = stats

# Display results
for group_name, stats in group_stats.items():
    print(f"\nGroup: {group_name}")
    print(stats)


Group: convnext_trades_v2
       metric     mean  std_dev      min      max  count
0   clean_acc 0.662035 0.253751 0.075333 0.982000     77
1  robust_acc 0.394740 0.259645 0.000667 0.931333     77

Group: convnext_classic_at
       metric     mean  std_dev      min      max  count
0   clean_acc 0.620166 0.288749 0.063333 0.983333     71
1  robust_acc 0.387489 0.291329 0.000000 0.929333     71

Group: vit_trades_v2
       metric     mean  std_dev      min      max  count
0   clean_acc 0.586718 0.294651 0.010000 0.960000     78
1  robust_acc 0.306639 0.267958 0.002667 0.890000     78

Group: vit_classic_at
       metric     mean  std_dev      min      max  count
0   clean_acc 0.476320 0.310968 0.006667 0.962000     77
1  robust_acc 0.313797 0.273216 0.000000 0.904667     77
